In [12]:
from numpy import *
from scipy.sparse import *

In [2]:
trackArtist=load("track_artist.npy")
trackArtist

array([[2972914,     144],
       [2750239,     246],
       [1550729,     144],
       ..., 
       [2584455,   27938],
       [2299706,  373892],
       [2739985,  567363]], dtype=int32)

In [6]:
trackArtist.shape

(100000, 2)

In [13]:
playlistTracks=load_npz("all_playlists_with_tracks.npz")
playlistTracks

<45649x200 sparse matrix of type '<class 'numpy.int64'>'
	with 1086171 stored elements in Compressed Sparse Column format>

In [14]:
playlistTracks.shape

(45649, 200)

In [41]:
playlistTracks[:,1:][1].data.shape

(11,)

In [84]:
myList=[x.data.tolist() for x in playlistTracks[:,1:]]

In [85]:
backupList=myList.copy()

In [87]:
len(myList)

45649

In [86]:
#myList[0]#queste erano le tracce

[2634448, 2693660, 2227105, 2654612, 162463, 1195736, 2861386, 795606, 421750]

In [88]:
tIDs=trackArtist[:,0]
tIDs

array([2972914, 2750239, 1550729, ..., 2584455, 2299706, 2739985], dtype=int32)

In [89]:
artistIDs=trackArtist[:,1]
artistIDs

array([   144,    246,    144, ...,  27938, 373892, 567363], dtype=int32)

In [97]:
for i,tracks in enumerate(myList):
    if(i%10000==0):
        print(tracks)
    myList[i]=list(map(lambda x: artistIDs[where(tIDs==x)[0][0]], tracks ))

[2634448, 2693660, 2227105, 2654612, 162463, 1195736, 2861386, 795606, 421750]
[3435317]
[1891218, 234607, 1966460, 3830948, 1838617, 2988391, 356765, 3701295, 656735, 2712377, 3674375]
[20355, 1232476]
[3382133, 2838946, 1165920, 3117823, 1079342]


In [106]:
#myList[0]#tracce trasformate in artisti

[20683, 281045, 22096, 281045, 319024, 319024, 449220, 54765, 53362]

In [109]:
paddedList=list(map(lambda x: x+[0]*(199-len(x)) , myList))

In [114]:
all_playlists_with_artist=csc_matrix(paddedList)

In [116]:
all_playlists_with_artist

<45649x199 sparse matrix of type '<class 'numpy.int64'>'
	with 1040522 stored elements in Compressed Sparse Column format>

In [117]:
save_npz("all_playlists_with_artist.npz",all_playlists_with_artist)